# Water Pump
### Register for Google Elevation API as API key is required to run this notebook

In [23]:
import pandas as pd
import matplotlib
import seaborn
import requests
import itertools
import numpy as np
import re
import json
from pandas.io.json import json_normalize

In [24]:
pumpValues = pd.read_csv('data.csv', index_col='id')

In [25]:
pumpResponse = pd.read_csv('response.csv', index_col='id')

In [26]:
main = pumpValues.join(pumpResponse)

In [28]:
main.shape

(59400, 40)

### Preprocessing
#### Data Cleaning
Marked missing or incorrect data as 'NA'
#### Fetch Missing Altitude Data
Uses Google's Elevation API to fetch missing water point altitude data (requires API Key).
#### Introduce Few Features
Added funder and installer types as well as a categorical column to check whether funder is installer

In [29]:
# Column types
dtypes = {
        'funder' : str,
        'installer' : str,
        'wpt_name' : str,
        'basin' : str,
        'subvillage' : str,
        'region' : str,
        'lga' : str,
        'ward' : str,
        'public_meeting' : str,
        'scheme_management' : str,
        'scheme_name' : str,
        'permit' : str,
        'extraction_type' : str,
        'extraction_type_group' : str,
        'extraction_type_class' : str,
        'management' : str,
        'management_group' : str,
        'payment_type' : str,
        'payment' : str,
        'water_quality' : str,
        'quality_group' : str,
        'source_type' : str,
        'source' : str,
        'quantity' : str,
        'quantity_group': str,
        'source_class' : str,
        'waterpoint_type_group' : str,
        'waterpoint_type' : str,
        'status_group' : str
}

ngos = ['acra','accord','OXFAM','AMREF','WATERAID','ngo']
intaid = ['unice','NORAD','JAICA','JICA','HIFAB','DANIDA','UNICEF']
african = ['afri','ADB','African Development Bank']
intgov = ['HESAWA','british','germany','netherland','belgium','norway','finland']
worldbank = ['RWSSP','RUDEP','WORLD BANK','RURAL WATER SUPPLY AND SANITAT']
institution = ngos + intaid + worldbank + ['TASAF','TAASAF'] 
gover = ['WSDP','DWSP','gove','DISTRICT','ministry','TANZANIA']
church = ['christ','WORLD VISION','AICT','ADRA','KKKT','chur','bapt','MISSION','CATHO','ROMAN']

def split_seq(iterable, size):
    it = iter(iterable)
    item = list(itertools.islice(it, size))
    while item:
        yield item
        item = list(itertools.islice(it, size))    


def contains(x, orgs):
    for org in orgs:
        if re.search(org,x, re.IGNORECASE):
            return True
    return False
  
def is_funder_installer(row):
    funder = row['funder']
    installer = row['installer']
    if funder == 'NA' or installer == 'NA':
        return 'NA'
    elif funder[:4] == installer[:4]:
        return 'Y'
    else:
        return 'N'

def categorize_installer(x):
    if contains(x, gover):
        return 'GOVER'
    elif contains(x, african):
        return 'AFRICA'
    elif contains(x, intgov):
        return 'INTGOV'
    elif contains(x, church):
        return 'CHURCH'
    elif re.search('private', x, re.IGNORECASE):
        return 'PRIVATE'
    elif x =='GO' or contains(x,institution):
        return 'INTAID'
    else:
        return 'OTHER'
    
def populate_height(main_df, key):
    """ 
    Uses Google's Elevation API to populate gps_height from latitude and longitude.
    main_df = Main DataFrame
    key = Google API Key
    """    
    print(main_df.gps_height.apply(np.isnan).sum())
    no_alt_df = main_df[main_df.gps_height == 0]
    print(no_alt_df.shape)
    a = zip(no_alt_df['latitude'],no_alt_df['longitude'])
    lat_long_lst = list(split_seq(a,300))
    print('Total {} Google Elevation API calls will be made'.format(len(lat_long_lst)))
    frames = []
    for locs in lat_long_lst:
        locations = ''
        for lat,long in locs:
            locations += str(round(lat,8)) + ',' + str(round(long, 8)) + '|'
        payload = 'locations='+ locations[:-1] +'&key=' + key
        url = 'https://maps.googleapis.com/maps/api/elevation/json'
        res = requests.get(url, params=payload)
        if res.status_code == requests.codes.ok:
            first_response_json = res.json()
            frames.append(json_normalize(first_response_json['results']))

    alt_df = pd.concat(frames)
    alt_df.index = no_alt_df.index
    main_df['gps_height'] = main_df.gps_height.combine(alt_df['elevation'].apply(round), lambda x,y: y if x == 0 else x)
    assert main_df[main_df.gps_height == 0].shape[0] == 0
    print(main_df.gps_height.apply(np.isnan).sum())
    return main_df


def preprocess(main, api_key,is_train=True):
    # Remove incorrect lat/long data
    if is_train:
        main = main[main.longitude != 0]
    # Populate gps_height
    main = populate_height(main, api_key)
    # To Upper
    for key, val in dtypes.items():
        if val is str:
            main[key] = main[key].str.upper()
    # Remove NaN
    na_values = { 
                'installer':'NA',
                'funder':'NA',
                'quantity_group':'NA',
                'quantity':'NA',
                'basin' :'NA', 
                'payment_type':'NA', 
                'public_meeting':'NA', 
                'permit':'NA', 
                'region' :'NA', 
                'lga':'NA', 
                'ward':'NA', 
                'subvillage' :'NA', 
                'scheme_management':'NA', 
                'scheme_name':'NA'}
    main = main.fillna(value=na_values)
    # Set 0s are NA in String columns
    for key, val in dtypes.items():
        if val is str:
            main[key] = main[key].apply(lambda x: 'NA' if x == '0' else x)
    # Add funder_type
    main['funder_type'] = main.funder.apply(categorize_installer)
    # Add installer type
    main['installer_type'] = main.installer.apply(categorize_installer)
    # Add funder_is_installer
    main['funder_is_installer'] = main[['funder','installer']].apply(is_funder_installer, axis=1)

    return main

In [30]:
main = preprocess(main=main,api_key='')

0
(18626, 40)
Total 63 Google Elevation API calls will be made


/Users/Zuber/anaconda/lib/python3.5/site-packages/ipykernel/__main__.py:77: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
/Users/Zuber/anaconda/lib/python3.5/site-packages/ipykernel/__main__.py:92: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy


0


In [31]:
main.to_csv('final_data.csv')

In [32]:
def make_categories(df, cols_to_convert):
    for col in cols_to_convert:
        df[col + '_cat'] = df[col].astype("category")
    return df

Index(['amount_tsh', 'date_recorded', 'funder', 'gps_height', 'installer',
       'longitude', 'latitude', 'wpt_name', 'num_private', 'basin',
       'subvillage', 'region', 'region_code', 'district_code', 'lga', 'ward',
       'population', 'public_meeting', 'recorded_by', 'scheme_management',
       'scheme_name', 'permit', 'construction_year', 'extraction_type',
       'extraction_type_group', 'extraction_type_class', 'management',
       'management_group', 'payment', 'payment_type', 'water_quality',
       'quality_group', 'quantity', 'quantity_group', 'source', 'source_type',
       'source_class', 'waterpoint_type', 'waterpoint_type_group',
       'status_group', 'funder_type', 'installer_type', 'funder_is_installer'],
      dtype='object')